# Highways that are Transit Deserts

These highway corridors have a dearth of bus transit service. 

## Criteria
* 10-mile corridors 
* No transit routes are found within 2 miles of the highway.

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

from shared_utils import geography_utils

import E1_get_buses_on_shn as get_buses_on_shn
import E4_highway_segments_stats as highway_utils

catalog = intake.open_catalog("./*.yml")

In [ ]:
def sjoin_hwy_segments_to_buses(segment_length: int): 
    highway_segments = get_buses_on_shn.cut_highway_segments(
        segment_length)
    
    # Get a 2 mile buffer around highway 10 mi segments
    highway_polygon = highway_utils.draw_buffer(
        highway_segments, 
        geography_utils.FEET_PER_MI * 2)
    
    bus_routes = catalog.bus_routes_all_aggregated_stats.read().to_crs(
        geography_utils.CA_StatePlane)
    
    # left join might result in some segments that join on 
    # certain bus routes but not others
    hwys_with_buses = gpd.sjoin(
        highway_polygon, 
        bus_routes, 
        how = "inner", 
        predicate = "intersects"
    ).drop(columns = "index_right")
    
    # Do a merge this way and keep left_only to be safe
    highway_segments2 = pd.merge(
        highway_segments,
        hwys_with_buses[["hwy_segment_id"]].drop_duplicates(),
        how = "left",
        validate = "1:1",
        indicator=True
    )
    
    no_buses = highway_segments2[highway_segments2._merge=="left_only"]

    no_buses = no_buses.assign(
        length = no_buses.geometry.length
    )
    
    # Remove short segments, which could occur if 
    # Route, RouteType change, and it doesn't get cut the full length
    # should be at least 75% of the segment length, 
    # so for a 10 mi corridor, corridor should be 7.5 mi long
    no_buses2 = (no_buses[no_buses.length >= segment_length * 0.75]
                 .drop(columns = ["length", "_merge"])
                 .reset_index(drop=True)
                )
    
    return no_buses2

In [ ]:
SEGMENT_DISTANCE = geography_utils.FEET_PER_MI * 10
gdf = sjoin_hwy_segments_to_buses(SEGMENT_DISTANCE)

#gdf = gdf.assign(
#    geometry = gdf.geometry.buffer(300)
#).to_crs(geography_utils.WGS84)

In [ ]:
def make_district_map(gdf: gpd.GeoDataFrame, district: int): 
    gdf2 = gdf[gdf.District==district].drop(
        columns = ["hwy_segment_id", "segment_sequence"])
    
    m = gdf2.explore("Route", 
                     categorical = True, tiles = "CartoDB Positron"
                    )
    
    display(m)

## District Maps

In [ ]:
districts = gdf.District.unique().tolist()

for i in sorted(districts):
    display(HTML(f"<h3>District {i}</h3>"))
    
    make_district_map(gdf, i)

<a id="top"></a>       